In [153]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from itertools import product
from pprint import pprint

In [154]:
url = "https://en.wikipedia.org/wiki/List_of_best-selling_video_games"
page = requests.get(url)
soup = BeautifulSoup(page.content.decode('utf-8'), 'html.parser')
print(f"The encoding is {page.encoding}")

The encoding is UTF-8


In [155]:
tb = soup.find_all("table")[1]
print(tb.encode('utf-8', errors='ignore'))

b'<table class="wikitable plainrowheaders sortable">\n<tbody><tr>\n<th scope="col">Title\n</th>\n<th scope="col">Sales\n</th>\n<th scope="col">Series\n</th>\n<th scope="col">Platform(s)\n</th>\n<th scope="col">Initial release date\n</th>\n<th scope="col">Developer(s)<sup class="reference" id="cite_ref-devpub_3-0"><a href="#cite_note-devpub-3">[b]</a></sup>\n</th>\n<th scope="col">Publisher(s)<sup class="reference" id="cite_ref-devpub_3-1"><a href="#cite_note-devpub-3">[b]</a></sup>\n</th>\n<th class="unsortable" scope="col"><abbr title="Reference(s)">Ref.</abbr>\n</th></tr>\n<tr>\n<td><i><a href="/wiki/Minecraft" title="Minecraft">Minecraft</a></i>\n</td>\n<td>300,000,000\n</td>\n<td><i><a href="/wiki/Minecraft_(franchise)" title="Minecraft (franchise)">Minecraft</a></i>\n</td>\n<td>Multi-platform\n</td>\n<td><span data-sort-value="000000002011-11-18-0000" style="white-space:nowrap">November 18, 2011</span><sup class="reference" id="cite_ref-6"><a href="#cite_note-6">[c]</a></sup>\n</t

In [156]:
def table_parsing(table_tag):
    rowspans = [] 
    rows = table_tag.find_all('tr')

    colcount = 0
    for r, row in enumerate(rows):
        cells = row.find_all(['td', 'th'], recursive = False)
        colcount = max(
            colcount,
            sum(int(c.get('colspan', 1)) or 1 for c in cells[:-1]) + len(cells[-1:]) + len(rowspans))
        rowspans += [int(c.get('rowspan', 1)) or len(rows) - r for c in cells]
        rowspans = [s - 1 for s in rowspans if s > 1]

    
    table = [[None] * colcount for row in rows]

    rowspans = {}  
    for row, row_elem in enumerate(rows):
        span_offset = 0   
        for col, cell in enumerate(row_elem.find_all(['td', 'th'], recursive=False)):
            col += span_offset
            while rowspans.get(col, 0):
                span_offset += 1
                col += 1

            rowspan = rowspans[col] = int(cell.get('rowspan', 1)) or len(rows) - row
            colspan = int(cell.get('colspan', 1)) or colcount - col
            span_offset += colspan - 1
            value = cell.get_text().strip()
            for drow, dcol in product(range(rowspan), range(colspan)):
                try:
                    table[row + drow][col + dcol] = value
                    rowspans[col + dcol] = rowspan
                except IndexError:
                    pass

        rowspans = {c: s - 1 for c, s in rowspans.items() if s > 1}

    return table

In [157]:
df_list = table_parsing(tb)
df_list

[['Title',
  'Sales',
  'Series',
  'Platform(s)',
  'Initial release date',
  'Developer(s)[b]',
  'Publisher(s)[b]',
  'Ref.'],
 ['Minecraft',
  '300,000,000',
  'Minecraft',
  'Multi-platform',
  'November 18, 2011[c]',
  'Mojang Studios',
  'Mojang Studios',
  '[4]'],
 ['Grand Theft Auto V',
  '190,000,000',
  'Grand Theft Auto',
  'Multi-platform',
  'September 17, 2013',
  'Rockstar North',
  'Rockstar Games',
  '[5]'],
 ['Tetris (EA)',
  '100,000,000',
  'Tetris',
  'Multi-platform',
  'September 12, 2006',
  'EA Mobile',
  'Electronic Arts',
  '[6]'],
 ['Wii Sports',
  '82,900,000',
  'Wii',
  'Wii',
  'November 19, 2006',
  'Nintendo EAD',
  'Nintendo',
  '[7]'],
 ['PUBG: Battlegrounds',
  '75,000,000',
  'PUBG Universe',
  'Multi-platform',
  'December 20, 2017',
  'PUBG Corporation',
  'Krafton',
  '[8]'],
 ['Mario Kart 8 / Deluxe',
  '65,470,000',
  'Mario Kart',
  'Wii U / Switch',
  'May 29, 2014',
  'Nintendo EAD / Nintendo EPD (Deluxe)',
  'Nintendo',
  '[d]'],
 ['Super

In [158]:
final_data = pd.DataFrame(df_list, columns = df_list[0])
final_data = final_data[1:]
final_data

,Title,Sales,Series,Platform(s),Initial release date,Developer(s)[b],Publisher(s)[b],Ref.
1,Minecraft,"300,000,000",Minecraft,Multi-platform,"November 18, 2011[c]",Mojang Studios,Mojang Studios,[4]
2,Grand Theft Auto V,"190,000,000",Grand Theft Auto,Multi-platform,"September 17, 2013",Rockstar North,Rockstar Games,[5]
3,Tetris (EA),"100,000,000",Tetris,Multi-platform,"September 12, 2006",EA Mobile,Electronic Arts,[6]
4,Wii Sports,"82,900,000",Wii,Wii,"November 19, 2006",Nintendo EAD,Nintendo,[7]
5,PUBG: Battlegrounds,"75,000,000",PUBG Universe,Multi-platform,"December 20, 2017",PUBG Corporation,Krafton,[8]
6,Mario Kart 8 / Deluxe,"65,470,000",Mario Kart,Wii U / Switch,"May 29, 2014",Nintendo EAD / Nintendo EPD (Deluxe),Nintendo,[d]
7,Super Mario Bros.,"58,000,000",Super Mario,Multi-platform,"September 13, 1985",Nintendo R&D4,Nintendo,[e]
8,Red Dead Redemption 2,"57,000,000",Red Dead,Multi-platform,"October 26, 2018",Rockstar Games,Rockstar Games,[5]
9,Overwatch,"50,000,000",Overwatch,Multi-platform,"May 24, 2016",Blizzard Entertainment,Blizzard Entertainment,[14]
10,The Witcher 3: Wild Hunt,"50,000,000",The Witcher,Multi-platform,"May 19, 2015",CD Projekt Red,CD Projekt,[15]


In [159]:
#clean the dataframe
games_df = final_data
games_df

,Title,Sales,Series,Platform(s),Initial release date,Developer(s)[b],Publisher(s)[b],Ref.
1,Minecraft,"300,000,000",Minecraft,Multi-platform,"November 18, 2011[c]",Mojang Studios,Mojang Studios,[4]
2,Grand Theft Auto V,"190,000,000",Grand Theft Auto,Multi-platform,"September 17, 2013",Rockstar North,Rockstar Games,[5]
3,Tetris (EA),"100,000,000",Tetris,Multi-platform,"September 12, 2006",EA Mobile,Electronic Arts,[6]
4,Wii Sports,"82,900,000",Wii,Wii,"November 19, 2006",Nintendo EAD,Nintendo,[7]
5,PUBG: Battlegrounds,"75,000,000",PUBG Universe,Multi-platform,"December 20, 2017",PUBG Corporation,Krafton,[8]
6,Mario Kart 8 / Deluxe,"65,470,000",Mario Kart,Wii U / Switch,"May 29, 2014",Nintendo EAD / Nintendo EPD (Deluxe),Nintendo,[d]
7,Super Mario Bros.,"58,000,000",Super Mario,Multi-platform,"September 13, 1985",Nintendo R&D4,Nintendo,[e]
8,Red Dead Redemption 2,"57,000,000",Red Dead,Multi-platform,"October 26, 2018",Rockstar Games,Rockstar Games,[5]
9,Overwatch,"50,000,000",Overwatch,Multi-platform,"May 24, 2016",Blizzard Entertainment,Blizzard Entertainment,[14]
10,The Witcher 3: Wild Hunt,"50,000,000",The Witcher,Multi-platform,"May 19, 2015",CD Projekt Red,CD Projekt,[15]


In [160]:
del games_df[games_df.columns[-1]]
games_df

,Title,Sales,Series,Platform(s),Initial release date,Developer(s)[b],Publisher(s)[b]
1,Minecraft,"300,000,000",Minecraft,Multi-platform,"November 18, 2011[c]",Mojang Studios,Mojang Studios
2,Grand Theft Auto V,"190,000,000",Grand Theft Auto,Multi-platform,"September 17, 2013",Rockstar North,Rockstar Games
3,Tetris (EA),"100,000,000",Tetris,Multi-platform,"September 12, 2006",EA Mobile,Electronic Arts
4,Wii Sports,"82,900,000",Wii,Wii,"November 19, 2006",Nintendo EAD,Nintendo
5,PUBG: Battlegrounds,"75,000,000",PUBG Universe,Multi-platform,"December 20, 2017",PUBG Corporation,Krafton
6,Mario Kart 8 / Deluxe,"65,470,000",Mario Kart,Wii U / Switch,"May 29, 2014",Nintendo EAD / Nintendo EPD (Deluxe),Nintendo
7,Super Mario Bros.,"58,000,000",Super Mario,Multi-platform,"September 13, 1985",Nintendo R&D4,Nintendo
8,Red Dead Redemption 2,"57,000,000",Red Dead,Multi-platform,"October 26, 2018",Rockstar Games,Rockstar Games
9,Overwatch,"50,000,000",Overwatch,Multi-platform,"May 24, 2016",Blizzard Entertainment,Blizzard Entertainment
10,The Witcher 3: Wild Hunt,"50,000,000",The Witcher,Multi-platform,"May 19, 2015",CD Projekt Red,CD Projekt


In [161]:
#change the names
games_df.rename(columns = {'Developer(s)[b]': 'Developer(s)', 'Publisher(s)[b]': 'Publisher(s)'}, inplace = True)
games_df

,Title,Sales,Series,Platform(s),Initial release date,Developer(s),Publisher(s)
1,Minecraft,"300,000,000",Minecraft,Multi-platform,"November 18, 2011[c]",Mojang Studios,Mojang Studios
2,Grand Theft Auto V,"190,000,000",Grand Theft Auto,Multi-platform,"September 17, 2013",Rockstar North,Rockstar Games
3,Tetris (EA),"100,000,000",Tetris,Multi-platform,"September 12, 2006",EA Mobile,Electronic Arts
4,Wii Sports,"82,900,000",Wii,Wii,"November 19, 2006",Nintendo EAD,Nintendo
5,PUBG: Battlegrounds,"75,000,000",PUBG Universe,Multi-platform,"December 20, 2017",PUBG Corporation,Krafton
6,Mario Kart 8 / Deluxe,"65,470,000",Mario Kart,Wii U / Switch,"May 29, 2014",Nintendo EAD / Nintendo EPD (Deluxe),Nintendo
7,Super Mario Bros.,"58,000,000",Super Mario,Multi-platform,"September 13, 1985",Nintendo R&D4,Nintendo
8,Red Dead Redemption 2,"57,000,000",Red Dead,Multi-platform,"October 26, 2018",Rockstar Games,Rockstar Games
9,Overwatch,"50,000,000",Overwatch,Multi-platform,"May 24, 2016",Blizzard Entertainment,Blizzard Entertainment
10,The Witcher 3: Wild Hunt,"50,000,000",The Witcher,Multi-platform,"May 19, 2015",CD Projekt Red,CD Projekt


In [162]:
# save our data to excel for future projects
games_df.to_excel("Game_data_parsing.xlsx")